In [3]:
import entity_tagging
import utils
import argparse
from datetime import datetime
import pandas as pd
import pickle
from sklearn.model_selection import KFold
import numpy as np

In [46]:
def load_corpus_xlsx(filename):
    corpus = pd.read_excel(filename, dtype={'msg': str})
    list_label = corpus['class'].to_list()
    list_content = corpus['msg'].to_list()
    return list_label, list_content

In [47]:
list_label, list_content = load_corpus_xlsx('../message_modified_v1.2.xlsx')

In [42]:
list_content = entity_tagging.entity_tagging(list_content)

# Begin test

In [44]:
# def test(list_content, test_content, list_label):
#     content = list_content.copy()
#     content.insert(0, test_content)
#     content = entity_tagging.entity_tagging(content)
#     # list_content_vec, list_len_sms, dictionary = utils.vectorize(content, 'bow')
#     list_content_vec, list_label, list_len_sms, dictionary = utils.doc_2_vec(content[:], list_label, 'bow')
#     clf = utils.build_classifier_svm(list_content_vec, list_label)
#     pickle.dump(clf, open(r'C:\Users\MINH.NV193012\Desktop\20222\Spam_detection_heroku_app\bow_svm_clf.pkl', 'wb'))
#
#     # Reads in saved classification model
#     list_content_vec, list_len_sms, dictionary = utils.vectorize(content, 'bow')
#     load_clf = pickle.load(open(r'C:\Users\MINH.NV193012\Desktop\20222\Spam_detection_heroku_app\bow_svm_clf.pkl', 'rb'))
#
#     prediction = load_clf.predict(list_content_vec[:1])
#
#     return prediction

In [65]:
test_content = 'Trung Tam Anh Ngu Hoa Ky Cleverlearn Vietnam(www.CLV.vn) MUNG NOEL: Dang ky hom nay-ZING qua lien tay-Nhan ngay HOC BONG len toi 2TRIEU DONG.Tel:0437261698'.lower()
list_label, list_content = load_corpus_xlsx('../message_modified_v1.2.xlsx')
list_content.insert(0, test_content)

In [66]:
list_content = entity_tagging.entity_tagging(list_content)

In [67]:
list_content_vec, list_len_sms, dictionary = utils.vectorize(list_content, 'bow')

In [69]:
list_content_vec.shape

(6311, 2066)

In [70]:
clf = utils.build_classifier_svm(list_content_vec, list_label)
pickle.dump(clf, open(r'C:\Users\MINH.NV193012\Desktop\20222\Spam_detection_heroku_app\bow_svm_clf.pkl', 'wb'))

ValueError: Found input variables with inconsistent numbers of samples: [6311, 6310]

In [71]:
load_clf = pickle.load(open(r'C:\Users\MINH.NV193012\Desktop\20222\Spam_detection_heroku_app\bow_svm_clf.pkl', 'rb'))

In [72]:
load_clf.predict(list_content_vec[:1])

ValueError: X has 2066 features, but LinearSVC is expecting 2063 features as input.

# End test

In [6]:
def train_evaluation(list_label, list_content, vectorize_method, classifier_list):

    if vectorize_method == 'bow':
        list_content_vec, list_label, list_len_sms, dictionary = utils.doc_2_vec(list_content, list_label, 'bow')
    for i in classifier_list:
        list_false_positive, list_false_negative, list_true_positive, list_true_negative = \
        utils.kfold_classification(list_content, list_content_vec, list_label, i, 5, vectorize_method)
        print('Evaluation for ' + i.upper())
        utils.evaluation(list_false_positive, list_false_negative, list_true_positive, list_true_negative)
        print('-------------------------------------------------')

In [39]:
def train_entire_dataset(list_content, list_label, vectorize_method):
    list_content_vec, list_label, list_len_sms, dictionary = utils.doc_2_vec(list_content, list_label, vectorize_method)
    clf = utils.build_classifier_svm(list_content_vec, list_label)
    pickle.dump(clf, open(r'C:\Users\MINH.NV193012\Desktop\20222\Spam_detection_heroku_app\bow_svm_clf.pkl', 'wb'))
    print('Traning Done!')

In [40]:
train_entire_dataset(list_content, list_label,'bow')

(6310, 2063)
[[ 4]
 [21]
 [ 7]
 ...
 [34]
 [24]
 [42]]
Dictionary<2062 unique tokens: ['như_cũ', 'vẫn', 'á', 'b', 'bên']...>
Traning Done!


In [9]:
train_evaluation(list_label, list_content, 'bow', 'svm')

Evaluation for SVM
False Positive Rate: 1.3351819116360293%
False Negative Rate: 18.79865045453338%
True Positive Rate: 81.20134954546663%
True Negative Rate: 98.66481808836397%
-------------------------------------------------
